In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.models import load_model
from sklearn.ensemble import VotingClassifier
import pickle
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score
import time

In [2]:
IoT = pd.read_csv('../../data/test_heterogeneous.csv')
IoT = IoT.iloc[0:5000]

IoT = IoT.dropna()

encoder=LabelEncoder()
IoT['fridge_temp_condition']=encoder.fit_transform(IoT['fridge_temp_condition'])
IoT['door_state']=encoder.fit_transform(IoT['door_state'])
IoT['door_sphone_signal']=encoder.fit_transform(IoT['door_sphone_signal'])
IoT['weather_type']=encoder.fit_transform(IoT['weather_type'])
IoT['ml_light_status']=encoder.fit_transform(IoT['ml_light_status'])

IoT.head()

,date-time,fridge_temperature,fridge_temp_condition,door_state,door_sphone_signal,gps_latitude,gps_longitude,modbus_FC1_Read_Input_Register,modbus_FC2_Read_Discrete_Value,modbus_FC3_Read_Holding_Register,modbus_FC4_Read_Coil,ml_motion_status,ml_light_status,thermostat_current_temperature,thermostat_status,weather_temperature,weather_pressure,weather_humidity,weather_type
0,31-03-19~12:36:52,13.10,0,0,0,0.0,10.0,11287.0,3537.0,27844.0,30527.0,1.0,1,25.0,0.0,31.788508,1.035,32.036579,2
1,31-03-19~12:36:53,8.65,0,0,0,0.0,10.0,11287.0,3537.0,27844.0,30527.0,0.0,0,25.0,0.0,41.630997,1.035,30.886165,2
2,31-03-19~12:36:54,2.00,1,0,0,0.0,10.0,11287.0,3537.0,27844.0,30527.0,0.0,0,25.0,0.0,42.256959,1.035,19.755908,2
3,31-03-19~12:36:55,4.80,1,0,0,0.0,10.0,53287.0,1463.0,33518.0,23014.0,1.0,1,25.0,0.0,49.116581,1.035,78.949621,2
4,31-03-19~12:36:56,10.70,0,0,0,0.0,10.0,53287.0,1463.0,33518.0,23014.0,1.0,1,25.0,0.0,24.017085,1.035,40.001059,2


In [3]:
x = IoT.drop(['weather_type', 'date-time'], axis=1)
y = IoT['weather_type']

features=len(x.columns)

x_train,x_test,y_train,y_test = train_test_split(x, y, test_size=0.2)

In [4]:
model3 = pickle.load(open('../h5s/random-forest.h5', 'rb'))
model4 = pickle.load(open('../h5s/logistic-regression.h5', 'rb'))
model5 = pickle.load(open('../h5s/linear-discriminant-analysis.h5', 'rb'))
model6 = pickle.load(open('../h5s/kNN.h5', 'rb'))
model7 = pickle.load(open('../h5s/CART.h5', 'rb'))
model8 = pickle.load(open('../h5s/support-vector-machine.h5', 'rb'))
model9 = pickle.load(open('../h5s/naive-bayes.h5', 'rb'))

In [5]:
voting = VotingClassifier(estimators=[('RF', model3), ('LR', model4), ('LDA', model5), ('KNN', model6), ('CART', model7), ('SVM', model8), ('Naive-Bayes', model9)], voting='hard')
voting.fit(x_train, y_train) 

for clf in (model3, model4, model5, model6, model7, model8, model9, voting):
    train_start_time=time.time()
    clf.fit(x_train, y_train)
    train_end_time=time.time()
    test_start_time=time.time()
    y_pred = clf.predict(x_test)
    test_end_time=time.time()
    print(clf.__class__.__name__, 'accuracy : ', accuracy_score(y_test, y_pred))
    print(clf.__class__.__name__, 'precision : ', precision_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred)))
    print(clf.__class__.__name__, 'recall : ', recall_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred)))
    print(clf.__class__.__name__, 'f1_score : ', f1_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred)))
    print(clf.__class__.__name__, 'train time : ', train_end_time-train_start_time, ' sec')
    print(clf.__class__.__name__, 'test time : ', test_end_time-test_start_time, ' sec')
    print('\n')

c:\Users\sharm\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


RandomForestClassifier accuracy :  0.983
RandomForestClassifier precision :  0.9831444668776601
RandomForestClassifier recall :  0.983
RandomForestClassifier f1_score :  0.982728075803979
RandomForestClassifier train time :  0.8774871826171875  sec
RandomForestClassifier test time :  0.024956941604614258  sec




c:\Users\sharm\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression accuracy :  0.606
LogisticRegression precision :  0.5500399011783825
LogisticRegression recall :  0.6158536585365854
LogisticRegression f1_score :  0.5801128116240233
LogisticRegression train time :  0.21541762351989746  sec
LogisticRegression test time :  0.002004861831665039  sec


LinearDiscriminantAnalysis accuracy :  0.822
LinearDiscriminantAnalysis precision :  0.8261252853721642
LinearDiscriminantAnalysis recall :  0.822
LinearDiscriminantAnalysis f1_score :  0.8232639387178707
LinearDiscriminantAnalysis train time :  0.014962196350097656  sec
LinearDiscriminantAnalysis test time :  0.001993417739868164  sec


KNeighborsClassifier accuracy :  0.511
KNeighborsClassifier precision :  0.5142680454723763
KNeighborsClassifier recall :  0.511
KNeighborsClassifier f1_score :  0.48868184315922053
KNeighborsClassifier train time :  0.003987312316894531  sec
KNeighborsClassifier test time :  0.11021900177001953  sec


DecisionTreeClassifier accuracy :  0.96
DecisionTree

c:\Users\sharm\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


VotingClassifier accuracy :  0.868
VotingClassifier precision :  0.8813390611894333
VotingClassifier recall :  0.868
VotingClassifier f1_score :  0.8656633175006061
VotingClassifier train time :  6.215677261352539  sec
VotingClassifier test time :  0.6434383392333984  sec




In [6]:
voting = VotingClassifier(estimators=[('SVM', model8), ('Naive-Bayes', model9), ('RF', model3), ('LR', model4), ('LDA', model5), ('KNN', model6), ('CART', model7)], voting='hard')
voting.fit(x_train, y_train) 

for clf in (model8, model9, model3, model4, model5, model6, model7, voting):
    train_start_time=time.time()
    clf.fit(x_train, y_train)
    train_end_time=time.time()
    test_start_time=time.time()
    y_pred = clf.predict(x_test)
    test_end_time=time.time()
    print(clf.__class__.__name__, 'accuracy : ', accuracy_score(y_test, y_pred))
    print(clf.__class__.__name__, 'precision : ', precision_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred)))
    print(clf.__class__.__name__, 'recall : ', recall_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred)))
    print(clf.__class__.__name__, 'f1_score : ', f1_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred)))
    print(clf.__class__.__name__, 'train time : ', train_end_time-train_start_time, ' sec')
    print(clf.__class__.__name__, 'test time : ', test_end_time-test_start_time, ' sec')
    print('\n')

c:\Users\sharm\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


SVC accuracy :  0.507
SVC precision :  0.6244305146959597
SVC recall :  0.6417721518987342
SVC f1_score :  0.5514928459520761
SVC train time :  3.5445632934570312  sec
SVC test time :  0.47870516777038574  sec


GaussianNB accuracy :  0.695
GaussianNB precision :  0.7566133611133794
GaussianNB recall :  0.695
GaussianNB f1_score :  0.6904430176517824
GaussianNB train time :  0.005986690521240234  sec
GaussianNB test time :  0.001991748809814453  sec


RandomForestClassifier accuracy :  0.982
RandomForestClassifier precision :  0.9821546709592928
RandomForestClassifier recall :  0.982
RandomForestClassifier f1_score :  0.9815533912292944
RandomForestClassifier train time :  1.4010884761810303  sec
RandomForestClassifier test time :  0.05884194374084473  sec




c:\Users\sharm\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression accuracy :  0.606
LogisticRegression precision :  0.5500399011783825
LogisticRegression recall :  0.6158536585365854
LogisticRegression f1_score :  0.5801128116240233
LogisticRegression train time :  0.5530457496643066  sec
LogisticRegression test time :  0.003987550735473633  sec


LinearDiscriminantAnalysis accuracy :  0.822
LinearDiscriminantAnalysis precision :  0.8261252853721642
LinearDiscriminantAnalysis recall :  0.822
LinearDiscriminantAnalysis f1_score :  0.8232639387178707
LinearDiscriminantAnalysis train time :  0.04089069366455078  sec
LinearDiscriminantAnalysis test time :  0.0049855709075927734  sec


KNeighborsClassifier accuracy :  0.511
KNeighborsClassifier precision :  0.5142680454723763
KNeighborsClassifier recall :  0.511
KNeighborsClassifier f1_score :  0.48868184315922053
KNeighborsClassifier train time :  0.010971307754516602  sec
KNeighborsClassifier test time :  0.25079917907714844  sec


DecisionTreeClassifier accuracy :  0.961
DecisionTree

c:\Users\sharm\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


VotingClassifier accuracy :  0.867
VotingClassifier precision :  0.8807269616062509
VotingClassifier recall :  0.867
VotingClassifier f1_score :  0.8645851913091562
VotingClassifier train time :  4.6131768226623535  sec
VotingClassifier test time :  1.398308515548706  sec




In [7]:
voting = VotingClassifier(estimators=[('LR', model4), ('SVM', model8), ('Naive-Bayes', model9), ('RF', model3), ('LDA', model5), ('KNN', model6), ('CART', model7)], voting='hard')
voting.fit(x_train, y_train) 

for clf in (model4, model8, model9, model3, model5, model6, model7, voting):
    train_start_time=time.time()
    clf.fit(x_train, y_train)
    train_end_time=time.time()
    test_start_time=time.time()
    y_pred = clf.predict(x_test)
    test_end_time=time.time()
    print(clf.__class__.__name__, 'accuracy : ', accuracy_score(y_test, y_pred))
    print(clf.__class__.__name__, 'precision : ', precision_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred)))
    print(clf.__class__.__name__, 'recall : ', recall_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred)))
    print(clf.__class__.__name__, 'f1_score : ', f1_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred)))
    print(clf.__class__.__name__, 'train time : ', train_end_time-train_start_time, ' sec')
    print(clf.__class__.__name__, 'test time : ', test_end_time-test_start_time, ' sec')
    print('\n')

c:\Users\sharm\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\sharm\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

LogisticRegression accuracy :  0.606
LogisticRegression precision :  0.5500399011783825
LogisticRegression recall :  0.6158536585365854
LogisticRegression f1_score :  0.5801128116240233
LogisticRegression train time :  0.4937098026275635  sec
LogisticRegression test time :  0.00997304916381836  sec


SVC accuracy :  0.507
SVC precision :  0.6244305146959597
SVC recall :  0.6417721518987342
SVC f1_score :  0.5514928459520761
SVC train time :  2.830934524536133  sec
SVC test time :  0.48624205589294434  sec


GaussianNB accuracy :  0.695
GaussianNB precision :  0.7566133611133794
GaussianNB recall :  0.695
GaussianNB f1_score :  0.6904430176517824
GaussianNB train time :  0.003988742828369141  sec
GaussianNB test time :  0.001994609832763672  sec


RandomForestClassifier accuracy :  0.982
RandomForestClassifier precision :  0.9821291704181673
RandomForestClassifier recall :  0.982
RandomForestClassifier f1_score :  0.9817274375308874
RandomForestClassifier train time :  1.564645528793335

c:\Users\sharm\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


VotingClassifier accuracy :  0.869
VotingClassifier precision :  0.8829560141045463
VotingClassifier recall :  0.869
VotingClassifier f1_score :  0.8670073730089455
VotingClassifier train time :  4.455155372619629  sec
VotingClassifier test time :  1.326552391052246  sec


